# Pose Landmarks Detection with MediaPipe Tasks
- mediapipe.solutions already provides a complete solution for pose detection
- with mediapipe.tasks specific settings can be made and the output can be customized

Documentation: https://developers.google.com/mediapipe/solutions/vision/pose_landmarker/python


## 1. Preparation
- Install mediapipe 
- Import necessary libraries
- Store Pose Landmarker model in variable

In [11]:
# Run command in terminal: python -m pip install mediapipe
import numpy as np
import cv2
import mediapipe as mp
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
from mediapipe.tasks import python
from mediapipe.tasks.python import vision


model_asset_path=r'mediapipe_models\pose_landmarker_heavy.task'

## 2. Function to Create Pose Landmarker

In [12]:
def draw_landmarks_on_image(rgb_image, detection_result):
  pose_landmarks_list = detection_result.pose_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected poses to visualize.
  for idx in range(len(pose_landmarks_list)):
    pose_landmarks = pose_landmarks_list[idx]

    # Draw the pose landmarks.
    pose_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    pose_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in pose_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      pose_landmarks_proto,
      solutions.pose.POSE_CONNECTIONS,
      solutions.drawing_styles.get_default_pose_landmarks_style())
  return annotated_image

## 3. Running inference and visualizing the results
- open webcam
- detect pose landmarks
- visualize pose landmarks


In [15]:
# Create an PoseLandmarker object
base_options = python.BaseOptions(model_asset_path)
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    output_segmentation_masks=True)
detector = vision.PoseLandmarker.create_from_options(options)

# Open Webcam
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Camera could not be opened.")
    exit()

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
        

    cv2.imshow('Webcam Feed', frame)
 
 
    # Detect pose landmarks from the input image.
    img = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
    detection_result = detector.detect(img)
    
    # Process the detection result. In this case, visualize it.
    img_annotated = draw_landmarks_on_image(img.numpy_view(), detection_result)
  
    # Display annotated image
    cv2.imshow('Annotated Webcam Feed', img_annotated)

    # Press 'q' to quit
    key = cv2.waitKey(1)
    if key != -1 or cv2.getWindowProperty('Webcam Feed', cv2.WND_PROP_VISIBLE) < 1:
        break

# Clean up
cap.release()
cv2.destroyAllWindows()
